In [1]:
import tensorflow.keras
from unet import unet_model_3d


In [2]:
input_shape = (1,20,20,20)
model = unet_model_3d(input_shape = input_shape, n_labels = 1, activation_name = 'relu')
#model.summary()

In [3]:
from data import get_train_and_validation_datasets

batch_size = 1
path_images = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_training/'
path_labels = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_targets/'

train_dataset, validation_dataset, validation_images = get_train_and_validation_datasets(0.2,path_images,path_labels,patch=True,patch_shape=(20,20,20))
train_dataset = train_dataset.shuffle(10).batch(batch_size).take(30)
validation_dataset = validation_dataset.batch(batch_size).take(30)


Number of images for training: 22
Number of images for validation: 6


In [4]:
for image in train_dataset:
    x,y = image
    print(x.shape)
    print(y.shape)
    break

(1, 1, 20, 20, 20)
(1, 1, 20, 20, 20)


In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from datetime import datetime

time = datetime.now().strftime('%Y-%m-%d_%H-%M')

model_checkpoint = ModelCheckpoint(filepath='models/best/' + time + '_model_weights.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   mode='max',
                                   save_best_only=True)

reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', 
                                      patience=10,
                                      mode='max', 
                                      verbose=1)

tensorboard_callback = TensorBoard(log_dir='runs/' + time,
                                   histogram_freq=3,
                                   write_grads=True)

callbacks = [model_checkpoint,reduce_lr_plateau,tensorboard_callback]


In [6]:
epochs = 6
model.fit(train_dataset,
          epochs=epochs,
          validation_data=validation_dataset,
          callbacks = callbacks)

Epoch 1/6
     30/Unknown - 34s 1s/step - loss: -0.2148 - dice_coefficient: 0.2148
Epoch 00001: val_loss improved from -inf to -0.96782, saving model to models/best/2020-03-08_08-59_model_weights.h5
30/30 [==============================] - 46s 2s/step - loss: -0.2148 - dice_coefficient: 0.2148 - val_loss: -0.9678 - val_dice_coefficient: 0.9678
Epoch 2/6
29/30 [============================>.] - ETA: 1s - loss: -1.0000 - dice_coefficient: 1.0000
Epoch 00002: val_loss did not improve from -0.96782
30/30 [==============================] - 41s 1s/step - loss: -1.0000 - dice_coefficient: 1.0000 - val_loss: -0.9684 - val_dice_coefficient: 0.9684
Epoch 3/6
 6/30 [=====>........................] - ETA: 34s - loss: -1.0000 - dice_coefficient: 1.0000WARNING:tensorflow:Can save best model only with val_loss available, skipping.


In [ ]:
model.save_weights("models/final/" + time + "_model_weights.h5")